# Crop Disease Identification Using CNN

## Objective
This project uses a Convolutional Neural Network (CNN) to classify crop leaf images as healthy or diseased.  
It is based on the PlantVillage dataset and supports deployment through a Flask web interface.

---

## 🗂️ Table of Contents
1. [Importing Libraries](#importing-libraries)
2. [Data Loading](#data-loading)
3. [Image Preprocessing](#image-preprocessing)
4. [CNN Model Building](#cnn-model-building)
5. [Model Training](#model-training)
6. [Evaluation](#evaluation)
7. [Model Saving](#model-saving)


## 1. Importing Libraries
## importing-libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


2025-06-30 22:47:35.952823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 2. Data Loading

In [4]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [8]:
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip -d data/
data_dir = 'data/PlantVillage'

Traceback (most recent call last):
  File "/opt/anaconda3/bin/kaggle", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open plantdisease.zip, plantdisease.zip.zip or plantdisease.zip.ZIP.


In [ ]:
data_dir = 'PlantVillage' 
img_height, img_width = 128, 128
batch_size = 32


## 3. 🧼 Image Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_generator.class_indices)
class_labels = list(train_generator.class_indices.keys())


## 4. 🧠 CNN Model Building

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 5. 🏋️‍♀️ Model Training

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stop]
)


## 6. 📈 Evaluation

In [ ]:
# Plot accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


## 7. 💾 Model Saving

In [ ]:
model.save('crop_disease_app/model/crop_model.h5')
print("Model saved successfully!")